In [ ]:
import numpy as np
import os
import base64
import re
import sqlite3
import time
from datetime import datetime
from flask import Flask, request, jsonify, g
from tensorflow.keras.models import load_model # load_model para cargar todo de una vez
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from PIL import Image
import io

In [ ]:
# Inicializamos la aplicación Flask
app = Flask(__name__)
# --- AJUSTE PARA AWS ---
application = app

# --- CONFIGURACIÓN ---
MODEL_VERSION = "1.0.0"
TARGET_SIZE = (265, 265)
LABELS = ["Clase_0", "Clase_1"]
DB_FILE = "hospital_data.db" # Archivo de base de datos local

# --- CARGA DEL MODELO (.keras) ---
# Ahora solo necesitamos un archivo único
MODEL_FILE = 'classifier-resnet-model9.keras'
model = None

try:
    if os.path.exists(MODEL_FILE):
        print(f"Cargando modelo desde {MODEL_FILE}...")
        # load_model se encarga de todo (arquitectura + pesos)
        model = load_model(MODEL_FILE)
        print("¡Modelo .keras cargado exitosamente!")
    else:
        print(f"ADVERTENCIA: No se encontró {MODEL_FILE}. La predicción fallará.")
except Exception as e:
    print(f"Error cargando modelo: {e}")

# --- GESTIÓN DE BASE DE DATOS (SQLite) ---

def get_db():
    """Conexión a base de datos por petición."""
    db = getattr(g, '_database', None)
    if db is None:
        db = g._database = sqlite3.connect(DB_FILE)
        db.row_factory = sqlite3.Row
    return db

@app.teardown_appcontext
def close_connection(exception):
    """Cierra la conexión al terminar la petición."""
    db = getattr(g, '_database', None)
    if db is not None:
        db.close()

def init_db():
    """Crea la tabla si no existe (se ejecuta al inicio)."""
    with app.app_context():
        db = get_db()
        db.execute('''
            CREATE TABLE IF NOT EXISTS predictions (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                date TEXT,
                filename TEXT,
                predicted_class TEXT,
                confidence REAL,
                corrected_label TEXT
            )
        ''')
        db.commit()

# Inicializamos la DB al arrancar el script
init_db()

# --- FUNCIONES AUXILIARES DE IMAGEN ---
def decode_base64_image(base64_string):
    image_data = re.sub('^data:image/.+;base64,', '', base64_string)
    image_bytes = base64.b64decode(image_data)
    return Image.open(io.BytesIO(image_bytes))

def prepare_image(image, target):
    if image.mode != "RGB":
        image = image.convert("RGB")
    image = image.resize(target)
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    image = preprocess_input(image)
    return image

# ==========================================
# DEFINICIÓN DE ENDPOINTS
# ==========================================

@app.route("/")
def home():
    return jsonify({
        "status": "online",
        "model_type": ".keras (Modern Format)",
        "endpoints": {
            "predict": "POST /predict (Body: image)",
            "history": "GET /history (Query: limit, class)",
            "train": "POST /admin/train (Body: epochs, lr)",
            "feedback": "PUT /feedback/<id> (Path + Body)"
        }
    })

# --- 1. RUTA CON PARÁMETROS EN EL BODY (Standard POST) ---
@app.route("/predict", methods=["POST"])
def predict():
    data = {"success": False}
    image_bytes = None
    filename = "upload_base64"

    # Lógica de recepción de imagen (Archivo o Base64)
    if request.files.get("image"):
        image_file = request.files["image"]
        filename = image_file.filename
        image_bytes = Image.open(io.BytesIO(image_file.read()))
    elif request.json and "image" in request.json:
        image_bytes = decode_base64_image(request.json["image"])

    if image_bytes and model:
        # 1. Predecir
        processed_image = prepare_image(image_bytes, target=TARGET_SIZE)
        preds = model.predict(processed_image)
        pred_idx = np.argmax(preds, axis=1)[0]
        prob = float(np.max(preds))
        pred_label = LABELS[pred_idx]

        # 2. Guardar en Base de Datos
        db = get_db()
        cursor = db.cursor()
        cursor.execute(
            'INSERT INTO predictions (date, filename, predicted_class, confidence) VALUES (?, ?, ?, ?)',
            (datetime.now().isoformat(), filename, pred_label, prob)
        )
        db.commit()
        prediction_id = cursor.lastrowid

        # 3. Responder
        data.update({
            "prediction_id": prediction_id,
            "prediction": pred_label,
            "confidence": f"{prob:.2%}",
            "success": True
        })
        return jsonify(data)

    return jsonify({"error": "Falta imagen o modelo no cargado"}), 400

# --- 2. RUTA CON PARÁMETROS EN LA QUERY (Query Params) ---
@app.route("/history", methods=["GET"])
def get_history():
    db = get_db()
    limit = request.args.get('limit', 10)
    class_filter = request.args.get('class_filter')

    query = "SELECT * FROM predictions"
    params = []

    if class_filter:
        query += " WHERE predicted_class = ?"
        params.append(class_filter)

    query += " ORDER BY id DESC LIMIT ?"
    params.append(limit)

    cursor = db.execute(query, params)
    rows = cursor.fetchall()

    history = [dict(row) for row in rows]

    return jsonify({"count": len(history), "data": history})

# --- 3. RUTA CON PARÁMETROS EN EL PATH + BODY (Combinado) ---
@app.route("/feedback/<int:prediction_id>", methods=["PUT"])
def update_feedback(prediction_id):
    if not request.json or 'correct_label' not in request.json:
        return jsonify({"error": "Falta 'correct_label' en el JSON body"}), 400

    new_label = request.json['correct_label']

    if new_label not in LABELS:
        return jsonify({"error": f"Clase inválida. Use: {LABELS}"}), 400

    db = get_db()
    cursor = db.execute("SELECT id FROM predictions WHERE id = ?", (prediction_id,))
    if not cursor.fetchone():
        return jsonify({"error": "ID de predicción no encontrado"}), 404

    db.execute(
        "UPDATE predictions SET corrected_label = ? WHERE id = ?",
        (new_label, prediction_id)
    )
    db.commit()

    return jsonify({"message": f"Feedback guardado para predicción {prediction_id}", "status": "updated"})

# --- 4. RUTA DE NEGOCIO AVANZADA (Simulacro de Entrenamiento) --- NO RECOMENDADO XQ PUEDE TIRAR AWS free
@app.route("/admin/train", methods=["POST"])
def train_model():
    params = request.json or {}
    epochs = params.get("epochs", 5)
    learning_rate = params.get("learning_rate", 0.001)

    print(f"Iniciando entrenamiento: Epochs={epochs}, LR={learning_rate}")
    time.sleep(2)

    global MODEL_VERSION
    MODEL_VERSION = f"2.0.{int(time.time())}"

    return jsonify({
        "message": "Entrenamiento finalizado (Simulado)",
        "new_version": MODEL_VERSION,
        "config_used": {
            "epochs": epochs,
            "learning_rate": learning_rate,
            "architecture": "ResNet50 (.keras)"
        }
    })



Cargando modelo desde classifier-resnet-model9.keras...
¡Modelo .keras cargado exitosamente!


In [ ]:
# --- INICIO DEL SERVIDOR ---
if __name__ == "__main__":
    # IMPORTANTE: En la nube, no usamos puerto fijo 5000.
    # AWS nos asigna un puerto a través de una variable de entorno.
    port = int(os.environ.get('PORT', 5000))

    # debug=False para producción (¡muy importante por seguridad!)
    app.run(host='0.0.0.0', port=port, debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


1.	Health check — GET /health y GET /health/model Fundamental para deployments y monitoreo. La segunda verifica que el modelo esté cargado correctamente.
2.	2. Predicción — POST /predict Recibe la imagen (como archivo o base64), ejecuta inferencia, devuelve probabilidad y clasificación.
3.	4. Metadata del modelo — GET /model/info Devuelve versión del modelo, clases que detecta, tamaño de input esperado. Muy útil para debugging y para el frontend.
